# 유튜브 랭킹 데이터 수집해서 시각화하기

In [1]:
!sudo apt-get install -y fonts-nanum > /dev/null
!sudo fc-cache -fv > /dev/null
!rm ~/.cache/matplotlib -rf > /dev/null

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [2]:
!apt-get update > /dev/null
!pip install selenium > /dev/null
!apt install chromium-chromedriver > /dev/null

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [7]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')   # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('chromedriver', options=options)

In [8]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=1'
driver.get(url)
# time.sleep(2)   # 로컬에서 이걸 안하면 .py에서 에러가 난다

In [11]:
trs = driver.find_elements_by_css_selector('.aos-init')
len(trs)

100

In [12]:
html =  driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [13]:
# soup.select('form > table > tbody > tr')
channel_list = soup.select('tr.aos-init')
len(channel_list)

100

In [23]:
channel = channel_list[0]
category = channel.select_one('p.category').get_text().strip(' \n[]')
category

'음악/댄스/가수'

In [22]:
name = channel.select_one('.subject a').text.strip()
name

'BLACKPINK'

In [25]:
subscriber = channel.select_one('.subscriber_cnt').text
view = channel.select_one('.view_cnt').text
video = channel.select_one('.video_cnt').text[:-1]
subscriber, view, video

('6400만', '190억0381만', '371')

In [27]:
channels = []

for channel in channel_list:
    category = channel.select_one('p.category').get_text().strip(' \n[]')
    name = channel.select_one('.subject a').text.strip()
    subscriber = channel.select_one('.subscriber_cnt').text
    view = channel.select_one('.view_cnt').text
    video = channel.select_one('.video_cnt').text[:-1]
    channels.append([category, name, subscriber, view, video])

In [28]:
df = pd.DataFrame(channels, columns=['카테고리','채널명','구독자수','조회수','비디오수'])
df.head()

,카테고리,채널명,구독자수,조회수,비디오수
0,음악/댄스/가수,BLACKPINK,6400만,190억0381만,371
1,음악/댄스/가수,HYBE LABELS,6030만,187억1305만,654
2,음악/댄스/가수,BANGTANTV,5640만,121억9496만,"1,579"
3,음악/댄스/가수,SMTOWN,2850만,218억7092만,"3,729"
4,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],2650만,110억5288만,223


In [29]:
df.tail()

,카테고리,채널명,구독자수,조회수,비디오수
95,음식/요리/레시피,까니짱 [ G-NI ],408만,10억3471만,482
96,키즈/어린이,로미유 스토리[Romiyu Story],408만,10억6557만,433
97,음악/댄스/가수,NCT DREAM,386만,3억7422만,228
98,BJ/인물/연예인,허팝Heopop,381만,31억3892만,"1,847"
99,애완/반려동물,크림히어로즈,381만,13억7401만,939


In [37]:
def convert_unit(s):
    if s.find('억'):
        s= s.replace('억','').replace('만', '0000')
        return s
    return s
    

In [36]:
df['조회수'] = df['조회수'].apply(lambda x : convert_unit(x)).astype(int).apply(lambda x : f'{x:,}')
df.head()

,카테고리,채널명,구독자수,조회수,비디오수
0,음악/댄스/가수,BLACKPINK,6400만,"19,003,810,000",371
1,음악/댄스/가수,HYBE LABELS,6030만,"18,713,050,000",654
2,음악/댄스/가수,BANGTANTV,5640만,"12,194,960,000","1,579"
3,음악/댄스/가수,SMTOWN,2850만,"21,870,920,000","3,729"
4,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],2650만,"11,052,880,000",223


- 두번째 페이지 -XPath로 찾아 Click()하여 이용하기

In [42]:
driver.find_element_by_xpath('//*[@id="list-skin"]/nav/span/a[1]').click()

In [43]:
html =  driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [44]:
channel_list = soup.select('tr.aos-init')

In [47]:
results = []

for channel in channel_list:
    category = channel.select_one('p.category').get_text().strip(' \n[]')
    name = channel.select_one('.subject a').text.strip()
    subscriber = channel.select_one('.subscriber_cnt').text
    view = channel.select_one('.view_cnt').text
    video = channel.select_one('.video_cnt').text
    results.append([category, name, subscriber, view, video])

In [48]:
df = pd.DataFrame(results, columns=['카테고리','채널명','구독자수','조회수','비디오수'])
df.head()

,카테고리,채널명,구독자수,조회수,비디오수
0,애완/반려동물,크림히어로즈,380만,13억7477만,940개
1,BJ/인물/연예인,워크맨-Workman,378만,6억7937만,141개
2,BJ/인물/연예인,waveya 2011,373만,13억4628만,702개
3,키즈/어린이,Lime Tube[라임튜브],373만,25억9542만,"1,723개"
4,음악/댄스/가수,WINNER,373만,11억7126만,428개


### page 1에서 10까지 클롤링하기

In [49]:
results = []

for page in range(1,11):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+str(page)
    driver.get(url)
    time.sleep(2)
    html =  driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    channel_list = soup.select('tr.aos-init')

    for channel in channel_list:
        category = channel.select_one('p.category').get_text().strip(' \n[]')
        name = channel.select_one('.subject a').text.strip()
        subscriber = channel.select_one('.subscriber_cnt').text
        view = channel.select_one('.view_cnt').text
        video = channel.select_one('.video_cnt').text
        results.append([category, name, subscriber, view, video])

In [50]:
df = pd.DataFrame(data=results, columns=['카테고리','채널명','구독자수','조회수','비디오수'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   카테고리    1000 non-null   object
 1   채널명     1000 non-null   object
 2   구독자수    1000 non-null   object
 3   조회수     1000 non-null   object
 4   비디오수    1000 non-null   object
dtypes: object(5)
memory usage: 39.2+ KB


In [51]:
df.tail()

,카테고리,채널명,구독자수,조회수,비디오수
995,게임,미소,51만,2억1077만,"3,084개"
996,미분류,밉지않은 관종언니,51만,6774만,170개
997,미분류,복지마블TV [Welfare Marble],51만,4996만,240개
998,음식/요리/레시피,Muggo,51만,1억8372만,"1,388개"
999,게임,임선비,51만,2억3429만,"1,134개"


In [60]:
import  numpy as np
df[df['채널명'].str.contains('희밥')].index

Int64Index([], dtype='int64')

In [55]:
df.to_csv('유튜브_순위csv', index=False)

In [58]:
driver.close()